In [21]:
import json

with open('../data/llm_dataset.json') as f:
    data = json.load(f)

In [25]:
from span_marker import SpanMarkerModel

# Download from the 🤗 Hub
# model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-bert-base-fewnerd-fine-super")
# model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-roberta-large-fewnerd-fine-super")
# model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-xlm-roberta-base-fewnerd-fine-super")

# model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-roberta-large-ontonotes5")

# model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-xlm-roberta-large-conll03") 
# model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-xlm-roberta-large-conll03-doc-context")
model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-xlm-roberta-large-conllpp-doc-context")

# Run inference
entities = model.predict(data[0])
entities


config.json:   0%|          | 0.00/3.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

This model was trained with document-level context: inference without document-level context may cause decreased performance.


[{'span': 'Brandon McLaughlin',
  'label': 'PER',
  'score': 0.9991335272789001,
  'char_start_index': 6,
  'char_end_index': 24},
 {'span': 'Chinese',
  'label': 'MISC',
  'score': 0.9989929795265198,
  'char_start_index': 50,
  'char_end_index': 57},
 {'span': 'NHS',
  'label': 'ORG',
  'score': 0.9431343078613281,
  'char_start_index': 76,
  'char_end_index': 79},
 {'span': 'Bitterne Surgery',
  'label': 'LOC',
  'score': 0.5204988718032837,
  'char_start_index': 167,
  'char_end_index': 183}]

## With spacy integration

In [11]:
import spacy

# Load the spaCy model with the span_marker pipeline component
nlp = spacy.load("en_core_web_lg", exclude=["ner"])
nlp.add_pipe("span_marker", config={"model": "tomaarsen/span-marker-roberta-large-ontonotes5"})

# Feed some text through the model to get a spacy Doc
doc = nlp(data[0])

# And look at the entities
print([(entity, entity.label_) for entity in doc.ents])

[(Brandon McLaughlin, 'PERSON'), (13-year-old, 'DATE'), (Chinese, 'NORP'), (NHS, 'ORG'), (568 968 0803, 'CARDINAL'), (1963-12-07, 'DATE'), (Bitterne Surgery, 'ORG'), (1976-06-03, 'DATE')]


In [18]:
data[0]

'\n\nMr. Brandon McLaughlin is a 13-year-old male of Chinese ethnicity with an NHS number of 568 968 0803 and a date of birth of 1963-12-07. He is single and is visiting Bitterne Surgery on 1976-06-03, with a diagnosis of acute bronchitis. His care plan includes respiratory therapy.'

In [19]:
import spacy
# spacy.cli.download('en_ner_bc5cdr_md')
# nlp = spacy.load("en_core_sci_scibert")

# Load the spaCy model with the span_marker pipeline component
nlp = spacy.load("en_core_sci_scibert", exclude=["ner"])
nlp.add_pipe("span_marker", config={"model": "tomaarsen/span-marker-roberta-large-ontonotes5"})

# Feed some text through the model to get a spacy Doc
doc = nlp(data[0])

# And look at the entities
print([(entity, entity.label_) for entity in doc.ents])

/Users/scarlettkynoch/miniconda3/envs/privacyfp/lib/python3.8/site-packages/spacy_transformers/layers/hf_shim.py:137: UserWarning: Error loading saved torch state_dict with strict=True, likely due to differences between 'transformers' versions. Attempting to load with strict=False as a fallback...

If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current 'transformers' and 'spacy-transformers' versions. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/scarlettkynoch/miniconda3/envs/privacyfp/lib/python3.8/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[(Brandon McLaughlin, 'PERSON'), (13-year-old, 'DATE'), (Chinese, 'NORP'), (NHS, 'ORG'), (568 968 0803, 'CARDINAL'), (1963-12-07, 'DATE'), (Bitterne Surgery, 'ORG'), (1976-06-03, 'DATE')]
